### Sales Drop - LTI Case Study code developed by 'Harish Visweswaraiya'

In [1]:
from IPython.core.display import display, HTML   #resize window
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell #pretty display
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import calendar
import numpy as np
pd.set_option('expand_frame_repr', False)

In [3]:
# Read data and create MonthName
df = pd.read_excel('./Case Study - Deep Dive Analysis.xlsx',sheet_name='input_data')
df.rename(columns={"Value Offtake(000 Rs)":"Value"}, inplace=True)
df["month_fmtd"] = df["month"].dt.month
df["year"] = df["month"].dt.year
df["month_fmtd"] = df['month_fmtd'].apply(lambda x: calendar.month_abbr[x])
df["MonthName"] = df["month_fmtd"].astype(str) + df["year"].astype(str)
df.drop(['month_fmtd','year','Item','PackSize','Packtype','month'], axis=1, inplace=True)

In [4]:
def prep(df, Manufacturer,target_period):
    df_tgt = df.loc[(df.Manufacturer == Manufacturer)&(df.MonthName==target_period)]
    Zone = df_tgt.Zone.unique().tolist()
    Region = df_tgt.Region.unique().tolist()
    Brand =  df_tgt.Brand.unique().tolist()
    Subbrand = df_tgt.Subbrand.unique().tolist()
    Dict = {'Zone': Zone, 'Region':Region, 'Brand':Brand, 'Subbrand':Subbrand}
    pd.DataFrame(Dict.items(), columns=['Level', 'Focus_Area'])
    df_dict = pd.DataFrame.from_dict(Dict, orient='index').reset_index()
    df_dict = df_dict.melt(id_vars=['index'],
        var_name="Level", 
        value_name="Focus_Area")
    df_dict.drop('Level',axis=1, inplace=True)
    df_dict.rename(columns={"index":"Level"}, inplace=True)
    df_dict = df_dict.dropna().reset_index(drop=True).sort_values('Level')
    df_dict['Manufacturer'] = Manufacturer
    cols = ['Manufacturer','Level','Focus_Area']
    df_dict = df_dict[cols]
    return df_dict

In [5]:
#Function to get results
def SalesDropAnalysis(Manufacturer, target_period, reference_period):
    df_tgt = df.loc[(df.Manufacturer == Manufacturer) & (df.MonthName== target_period)]
    df_ref = df.loc[(df.Manufacturer == Manufacturer) & (df.MonthName== reference_period)]
    tgt_period_sales = df_tgt.Value.sum()
    ref_period_sales = df_ref.Value.sum()
    
    if tgt_period_sales > ref_period_sales:
        return print("There is no drop in the sales for "+Manufacturer+" in "+target_period)
    else:
        df_new = prep(df, Manufacturer, target_period)
        ls_gr, ls_contr, ls_product = [], [], []
        for index, row in df_new.iterrows():
            Level, Focus_Area  = row['Level'], row['Focus_Area']
            tgt_gb = df_tgt.groupby(Level)
            tgt_fa_val = tgt_gb.aggregate(np.sum).loc[Focus_Area, :].at['Value']
            ref_gb = df_ref.groupby(Level)
            ref_fa_val = ref_gb.aggregate(np.sum).loc[Focus_Area, :].at['Value']
            gr = ((tgt_fa_val/ref_fa_val) - 1) * 100
            contr = (tgt_fa_val/tgt_period_sales) * 100
            product = gr * contr / 10000
            ls_gr.append(gr)
            ls_contr.append(contr)
            ls_product.append(product)
            
        df_new["growth_rate(%)"], df_new["contribution(%)"], df_new["product"]  = ls_gr, ls_contr, ls_product 
        df_new = df_new.sort_values(by='product').reset_index(drop=True)
        
        return print(df_new.round({"growth_rate(%)":0, "contribution(%)":0, "product":2}))

### Example 1

In [6]:
SalesDropAnalysis(Manufacturer='AMUL', target_period='May2019', reference_period='Apr2019')

  Manufacturer     Level     Focus_Area  growth_rate(%)  contribution(%)  product
0         AMUL     Brand           AMUL           -28.0            100.0    -0.28
1         AMUL  Subbrand  AMUL PRO WHEY           -28.0            100.0    -0.28
2         AMUL    Region          Urban           -28.0             89.0    -0.25
3         AMUL      Zone           East           -78.0              9.0    -0.07
4         AMUL      Zone           West           -11.0             37.0    -0.04
5         AMUL      Zone          South            -7.0             35.0    -0.02
6         AMUL    Region          Rural           -23.0             11.0    -0.02
7         AMUL      Zone          North            -3.0             19.0    -0.01


### Example 2

In [7]:
SalesDropAnalysis(Manufacturer='KRAFT FOODS', target_period='Apr2019', reference_period='Mar2019')

  Manufacturer     Level                Focus_Area  growth_rate(%)  contribution(%)  product
0  KRAFT FOODS    Region                     Urban           -11.0             89.0    -0.10
1  KRAFT FOODS     Brand                   COMPLAN            -8.0            100.0    -0.08
2  KRAFT FOODS  Subbrand  COMPLAN ROYALE CHOCOLATE            -8.0            100.0    -0.08
3  KRAFT FOODS      Zone                      East           -26.0             18.0    -0.05
4  KRAFT FOODS      Zone                      West            -7.0             35.0    -0.03
5  KRAFT FOODS      Zone                     South            -1.0             13.0    -0.00
6  KRAFT FOODS      Zone                     North             3.0             34.0     0.01
7  KRAFT FOODS    Region                     Rural            28.0             11.0     0.03


### Example 3

In [8]:
SalesDropAnalysis(Manufacturer='KRAFT FOODS', target_period='May2019', reference_period='Apr2019')

There is no drop in the sales for KRAFT FOODS in May2019
